In [1]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv

C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(dotenv_path='.env')

# https://platform.openai.com/api-keys
openai_api_key = os.getenv("OPENAI_API_KEY")

# https://console.groq.com/keys
groq_api_key = os.getenv("GROQ_API_KEY")

# https://console.anthropic.com/dashboard
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# https://aistudio.google.com/app/apikey
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [3]:
#!pip install pypdf
#!pip install faiss-cpu

from langchain_community.document_loaders.pdf import PyPDFLoader
loader = PyPDFLoader("dados/ExplorersGuide.pdf")
pages = loader.load_and_split()
print(len(pages))

#file_path = os.path.abspath("data/lei.pdf")
#if os.path.isfile(file_path):
#    print("ok")
#else:
#    print("erro")
#print(os.path.abspath(file_path))

49


In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
from langchain_community.vectorstores.faiss import FAISS 
db = FAISS.from_documents(pages, embeddings)

In [6]:
question = "Qual a cor da tunica tradicional do personagem Link?"
chunks = db.similarity_search(question)
len(chunks)

4

In [7]:
chunks[0]

Document(metadata={'source': 'dados/ExplorersGuide.pdf', 'page': 35}, page_content='While Link’s traditional green \ntunic is certainly an iconic look, his \nwardrobe has expanded quite a bit \nin his latest adventure. How you \ndress isn’t just an expression of \npersonal style either. In addition \nto upping your defense a bit, \ndifferent outﬁts have different \neffects that will prove useful \ndepending on what you’re doing \nand where you’re going. The Warm \nDoublet, for example, is a fantastic \nfusion of fashion and function. It not only looks rather spiffy, it also \nprotects you from the cold, allow-\ning you to explore snowy areas \nwithout losing health along the \nway. The Climber’s set ups your \nclimbing speed, which means \nyou’ll be able to climb farther than you normally would \nbefore running out of \nstamina. The Stealth set \nincreases your stealth, \nso it’s handy when \nyou’re out hunting or \ntrying to sneak up on \nenemies. The Zora \nArmor not only helps \nyou

In [8]:
chunks[0].page_content

'While Link’s traditional green \ntunic is certainly an iconic look, his \nwardrobe has expanded quite a bit \nin his latest adventure. How you \ndress isn’t just an expression of \npersonal style either. In addition \nto upping your defense a bit, \ndifferent outﬁts have different \neffects that will prove useful \ndepending on what you’re doing \nand where you’re going. The Warm \nDoublet, for example, is a fantastic \nfusion of fashion and function. It not only looks rather spiffy, it also \nprotects you from the cold, allow-\ning you to explore snowy areas \nwithout losing health along the \nway. The Climber’s set ups your \nclimbing speed, which means \nyou’ll be able to climb farther than you normally would \nbefore running out of \nstamina. The Stealth set \nincreases your stealth, \nso it’s handy when \nyou’re out hunting or \ntrying to sneak up on \nenemies. The Zora \nArmor not only helps \nyou swim faster, it also allows you \nto swim up waterfalls. Wearing a \ncomplete set 

In [11]:
question = "Qual a cor da tunica tradicional do personagem Link?"
db = FAISS.from_documents(pages, embeddings)
result = db.similarity_search_with_score(question, k=3)
for doc, score in result:
    print(f"Documento: {doc.page_content}\nRelevância: {score}\nMetadata: {doc.metadata}\n\n")


Documento: While Link’s traditional green 
tunic is certainly an iconic look, his 
wardrobe has expanded quite a bit 
in his latest adventure. How you 
dress isn’t just an expression of 
personal style either. In addition 
to upping your defense a bit, 
different outﬁts have different 
effects that will prove useful 
depending on what you’re doing 
and where you’re going. The Warm 
Doublet, for example, is a fantastic 
fusion of fashion and function. It not only looks rather spiffy, it also 
protects you from the cold, allow-
ing you to explore snowy areas 
without losing health along the 
way. The Climber’s set ups your 
climbing speed, which means 
you’ll be able to climb farther than you normally would 
before running out of 
stamina. The Stealth set 
increases your stealth, 
so it’s handy when 
you’re out hunting or 
trying to sneak up on 
enemies. The Zora 
Armor not only helps 
you swim faster, it also allows you 
to swim up waterfalls. Wearing a 
complete set with upgraded 
piec

In [9]:
contexto = chunks[0].page_content + chunks[1].page_content + chunks[2].page_content

In [10]:
template = """
Responda a pergunta {question} com base neste contexto: {context}
"""
from langchain.prompts.prompt import PromptTemplate
prompt = PromptTemplate(input_variables=['question','context'],template=template)
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)
chain = prompt | llm
response = chain.invoke({"question":question, "context":chunks[0].page_content})
print(response.content)


A cor da túnica tradicional do personagem Link é verde.
